In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from keras import layers

In [3]:
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

In [4]:
from openpyxl import Workbook
from openpyxl import load_workbook

In [5]:
import pathlib
from datetime import datetime

In [6]:
import itertools
import os

In [7]:
(train_ds, validation_ds, test_ds), info = tfds.load(
    "colorectal_histology",
    split=["train[:85%]", "train[85%:95%]", "train[95%:]"],
    with_info=True,
    as_supervised=True,
    shuffle_files= True,
)

num_classes = info.features['label'].num_classes

In [8]:
size = (150, 150)
batch_size = 1

train_ds = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
validation_ds = validation_ds.map(lambda x, y: (tf.image.resize(x, size), y))
test_ds = test_ds.map(lambda x, y: (tf.image.resize(x, size), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=1)
validation_ds = validation_ds.cache().batch(batch_size).prefetch(buffer_size=1)
test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=1)

In [11]:
%cd  /home/reynier/py_projects/Deployments/functions

/home/reynier/py_projects/Deployments/functions


In [12]:
from Nets import GetNeuralNetworkModel
from Stats import WeightQuantization, ActivationStats, CheckAccuracyAndLoss, QuantizationEffect, GetReadAndWrites
from FileAnalize import analize_file, analize_file_uno,analize_file_uno_ceros, save_file, load_file
from funciones import buffer_vectores
from Simulation import buffer_simulation, save_obj, load_obj

In [13]:
model = GetNeuralNetworkModel('Xception',(150,150,3),8, quantization = False, aging_active=False)

In [14]:
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [15]:
%cd  /home/reynier/py_projects/Deployments/Xception/Weights

/home/reynier/py_projects/Deployments/Xception/Weights


In [16]:
model.load_weights('weights.data')

In [43]:
threadsperblock = 512
ROWS      	   = 25
COLUMNS   	   = 25
acts_per_cycle = 25

def hardware_utilization(layer):
    parameters = {}
    if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D','SeparableConv2D']:
        map_size                       = layer.output_shape[1] if layer.__class__.__name__ == 'Conv1D' else layer.output_shape[1]*layer.output_shape[2]
        parameters['Procesing Cycles'] = np.prod(layer.kernel_size) if layer.__class__.__name__ == 'DepthwiseConv2D' else np.prod(layer.kernel_size)*layer.input_shape[-1]
        parameters['Columns Used']     = 1 if layer.__class__.__name__ == 'DepthwiseConv2D' else min(layer.filters, COLUMNS, parameters['Procesing Cycles'])
        parameters['Rows Used']        = min(ROWS, map_size)
        parameters['Initial Delay']    = parameters['Columns Used'] + parameters['Rows Used'] + 2
        parameters['Total cycles']     = int(parameters['Initial Delay'] + parameters['Procesing Cycles']*np.ceil(map_size/parameters['Rows Used'])*np.ceil(layer.output_shape[-1]/parameters['Columns Used']))							 
    else:
        parameters['Columns Used']     = min(layer.output_shape[-1],COLUMNS)
        parameters['Rows Used']        = min(int(np.ceil(layer.output_shape[-1]/parameters['Columns Used'])),ROWS)
        parameters['Initial Delay']    = 2
        parameters['Procesing Cycles'] = layer.input_shape[-1]
        parameters['Total cycles']     = int(parameters['Initial Delay'] + parameters['Procesing Cycles']*np.ceil(layer.output_shape[-1]/(parameters['Columns Used']*parameters['Rows Used'])))
    
    return parameters

In [68]:
Name = []
Type = []
Kernel = []
Shape = []
comp_utilization = []

Procesing_Cycles = []
Columns_Used = []
Rows_Used = []
Initial_Delay = []
Total_Cycles = []

for layer in model.layers:
    if layer.__class__.__name__ in ['Conv2D','Conv1D','DepthwiseConv2D','SeparableConv2D']:
        kernel = layer.kernel_size
        comp_utilization = hardware_utilization(layer)
    elif layer.__class__.__name__ in ['MaxPooling2D','Dense','Activation']:
        kernel = None
        comp_utilization = hardware_utilization(layer)
    else:
        kernel = None
        comp_utilization = None
        
    Name.append(layer.name)
    Type.append(layer.__class__.__name__)
    Kernel.append(kernel)
    Shape.append(layer.output_shape)

    if(comp_utilization != None):
        Procesing_Cycles.append(comp_utilization['Procesing Cycles'])
        Columns_Used.append(comp_utilization['Columns Used'])
        Rows_Used.append(comp_utilization['Rows Used'])
        Initial_Delay.append(comp_utilization['Initial Delay'])
        Total_Cycles.append(comp_utilization['Total cycles'])
    else:
        Procesing_Cycles.append(None)
        Columns_Used.append(None)
        Rows_Used.append(None)
        Initial_Delay.append(None)
        Total_Cycles.append(None)
    
    

Name_DF= pd.DataFrame(Name)
Type_DF= pd.DataFrame(Type)
Kernel_DF= pd.DataFrame(Kernel)
Shape_DF= pd.DataFrame(pd.array(Shape))
Procesing_Cycles_DF= pd.DataFrame(Procesing_Cycles)
Columns_Used_DF= pd.DataFrame(Columns_Used)
Rows_Used_DF= pd.DataFrame(Rows_Used)
Initial_Delay_DF= pd.DataFrame(Initial_Delay)
Total_Cycles_DF= pd.DataFrame(Total_Cycles)

model_layers = pd.concat([Name_DF, Type_DF, Kernel_DF, Shape_DF, Procesing_Cycles_DF, Columns_Used_DF, Rows_Used_DF, Initial_Delay_DF, Total_Cycles_DF], axis=1, join='outer')
model_layers.columns =['Name','Type','Kernel', 'Shape', 'Procesing Cycles', 'Columns Used','Rows Used', 'Initial Delay', 'Total Cycles']
model_layers.to_excel('Xception_model.xlsx', sheet_name='report', index=False)

print(' completada: ', datetime.now().strftime("%H:%M:%S"))
print(' completada: ', datetime.now().strftime("%H:%M:%S"))



 completada:  00:23:34
 completada:  00:23:34


/home/reynier/anaconda3/envs/tf_ml_projects/lib/python3.9/site-packages/pandas/core/arrays/numpy_.py:99: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(scalars, dtype=dtype)  # type: ignore[arg-type]
